In [1]:
import pandas as pd
from algos.daddy.live_trader import liveTrading

In [3]:
EXCHANGES = pd.concat([pd.read_csv('algos/daddy/exchanges.csv'), pd.read_csv('algos/eth_daddy/exchanges.csv')])
EXCHANGES = EXCHANGES[EXCHANGES['trade'] == 1]
EXCHANGES = EXCHANGES.drop_duplicates(subset=['exchange', 'symbol']) 
EXCHANGES = EXCHANGES.fillna("")

In [4]:
EXCHANGES

,name,exchange,types,name_type,cryptofeed_name,ccxt_symbol,symbol,cryptofeed_symbol,subaccount,trade,collateral,max_trade,buy_method,sell_method,html_color
0,bitmex,bitmex,book,fund,Bitmex,BTC/USD,XBTUSD,XBTUSD,,1,BTC,0.5,10sec_average,10sec_average,danger
0,eth_ftx,ftx,book,fund,FTX,ETH-PERP,ETHUSDT,ETH-PERP,daddy_ETH,1,USD,0.0,10sec_average,10sec_average,info


In [5]:
lt = liveTrading('ftx', 'eth_ftx', symbol='ETH-PERP', testnet=False, parameter_file='algos/eth_daddy/parameters.json', config_file='algos/eth_daddy/exchanges.csv')

In [6]:
lt.update_stop()

Removing stop at 1676.0 to add stop at 1606


In [8]:
import json

In [9]:
config = json.load(open("daddy.json"))


In [12]:
config['secondary_currency']['ETH']

'USD'

In [7]:
lt.set_position()

In [8]:
current_pos = lt.r.get('{}_current_pos'.format(lt.name)).decode()

In [9]:
current_pos

'LONG'

In [37]:
stop = lt.get_stop()

In [38]:
stop

['1676.0']

In [31]:
orders = lt.exchange.request('conditional_orders', api='private', method='GET', params={'market': lt.symbol_here})['result']

In [32]:
orders

[{'id': '39371244',
  'market': 'ETH-PERP',
  'future': 'ETH-PERP',
  'side': 'sell',
  'type': 'stop',
  'orderPrice': None,
  'triggerPrice': '1676.0',
  'size': '8.756',
  'status': 'open',
  'createdAt': '2021-05-04T05:45:27.844814+00:00',
  'triggeredAt': None,
  'orderId': None,
  'error': None,
  'reduceOnly': False,
  'trailValue': None,
  'trailStart': None,
  'cancelledAt': None,
  'cancelReason': None,
  'retryUntilFilled': True,
  'orderType': 'market',
  'filledSize': '0.0',
  'avgFillPrice': None}]

In [33]:
pos, entryPrice, amount = lt.get_position()

In [34]:
entryPrice

3352.8771128369117

In [35]:
close_at = int(entryPrice * lt.parameters['stop_percentage'])

In [36]:
close_at

1676

In [24]:
lt.close_stop_order()

In [25]:
current_pos, avgEntryPrice, amount = lt.get_position()

In [27]:
close_at = int(avgEntryPrice * lt.parameters['stop_percentage'])

In [28]:
close_at

1676

In [30]:
params = {
        'triggerPrice': close_at
    }

order = lt.exchange.create_order(lt.symbol, "stop", "sell", amount, None, params)